In [14]:
import onnx
import onnxruntime
import numpy as np
from onnx import shape_inference
from os import path
import sys
from tabulate import tabulate
from onnx import onnx_ml_pb2 as xpb2

In [15]:
model = onnx.load('./models/alexnet.onnx')
onnx.checker.check_model(model)
inferred_model = shape_inference.infer_shapes(model)
print('shape inference complete ...')

shape inference complete ...


### model characteristics

In [16]:
# Extract unique operator names
# Print the number of ONNX operators
num_operators = len(model.graph.node)
print(f"Number of ONNX operators: {num_operators}")

unique_operator_names = set(node.op_type for node in model.graph.node)
print("Unique ONNX operator names:")
for op_name in unique_operator_names:
    print(f"- {op_name}")

# Print attributes for each operator
print("\nAttributes for each operator:")
for node in model.graph.node:
    op_name = node.op_type
    print(f"\nOperator: {op_name}")

    for attr in node.attribute:
        print(f"{attr.name}: {attr.ints}")

Number of ONNX operators: 20
Unique ONNX operator names:
- AveragePool
- Conv
- Gemm
- Flatten
- MaxPool
- Relu

Attributes for each operator:

Operator: Conv
dilations: [1, 1]
group: []
kernel_shape: [11, 11]
pads: [2, 2, 2, 2]
strides: [4, 4]

Operator: Relu

Operator: MaxPool
ceil_mode: []
dilations: [1, 1]
kernel_shape: [3, 3]
pads: [0, 0, 0, 0]
strides: [2, 2]

Operator: Conv
dilations: [1, 1]
group: []
kernel_shape: [5, 5]
pads: [2, 2, 2, 2]
strides: [1, 1]

Operator: Relu

Operator: MaxPool
ceil_mode: []
dilations: [1, 1]
kernel_shape: [3, 3]
pads: [0, 0, 0, 0]
strides: [2, 2]

Operator: Conv
dilations: [1, 1]
group: []
kernel_shape: [3, 3]
pads: [1, 1, 1, 1]
strides: [1, 1]

Operator: Relu

Operator: Conv
dilations: [1, 1]
group: []
kernel_shape: [3, 3]
pads: [1, 1, 1, 1]
strides: [1, 1]

Operator: Relu

Operator: Conv
dilations: [1, 1]
group: []
kernel_shape: [3, 3]
pads: [1, 1, 1, 1]
strides: [1, 1]

Operator: Relu

Operator: MaxPool
ceil_mode: []
dilations: [1, 1]
kernel_sha

### Data bandwidth requirement

In [17]:
def _parse_element(elem: xpb2.ValueInfoProto):
    name = getattr(elem, 'name', "None")
    data_type = "NA"
    shape_str = "NA"
    etype = getattr(elem, 'type', False)
    if etype:
        ttype = getattr(etype, 'tensor_type', False)
        if ttype:
            data_type = getattr(ttype, 'elem_type', 0)
            shape = getattr(elem.type.tensor_type, "shape", False)
            if shape:
                shape_str = "["
                dims = getattr(shape, 'dim', [])
                for dim in dims:
                    vals = getattr(dim, 'dim_value', "?")
                    shape_str += (str(vals) + ",")
                shape_str = shape_str.rstrip(",")
                shape_str += "]"
    return name, data_type, shape_str

def get_valueproto_or_tensorproto_by_name(name: str, graph: xpb2.GraphProto):
    for i, node in enumerate(inferred_model.graph.node):
            if node.name == "":
                inferred_model.graph.node[i].name = str(i)
    input_nlist = [k.name for k in graph.input]
    initializer_nlist = [k.name for k in graph.initializer]
    value_info_nlist = [k.name for k in graph.value_info]
    output_nlist = [k.name for k in graph.output]

    # get tensor data
    if name in input_nlist:
        idx = input_nlist.index(name)
        return graph.input[idx], int(1)
    elif name in value_info_nlist:
        idx = value_info_nlist.index(name)
        return graph.value_info[idx], int(2)
    elif name in initializer_nlist:
        idx = initializer_nlist.index(name)
        return graph.initializer[idx], int(3)
    elif name in output_nlist:
        idx = output_nlist.index(name)
        return graph.output[idx], int(4)
    else:
        print("[ERROR MASSAGE] Can't find the tensor: ", name)
        print('input_nlist:\n', input_nlist)
        print('===================')
        print('value_info_nlist:\n', value_info_nlist)
        print('===================')
        print('initializer_nlist:\n', initializer_nlist)
        print('===================')
        print('output_nlist:\n', output_nlist)
        print('===================')
        return False, 0

def cal_tensor_mem_size(elem_type: str, shape: [int]):
    """ given the element type of the tensor and its shape, and return its memory size.

    Utility.

    Args:
        ttype: the type of the element of the given tensor. format: 'int', ...
        shape: the shape of the given tensor. format: [] of int

    Returns:
        mem_size: int
    """
    # init
    mem_size = int(1)
    # traverse the list to get the number of the elements
    for num in shape:
        mem_size *= num
    # multiple the size of variable with the number of the elements
    # "FLOAT": 1,
    # "UINT8": 2,
    # "INT8": 3,
    # "UINT16": 4,
    # "INT16": 5,
    # "INT32": 6,
    # "INT64": 7,
    # # "STRING" : 8,
    # "BOOL": 9,
    # "FLOAT16": 10,
    # "DOUBLE": 11,
    # "UINT32": 12,
    # "UINT64": 13,
    # "COMPLEX64": 14,
    # "COMPLEX128": 15
    if elem_type == 1:
        mem_size *= 4
    elif elem_type == 2:
        mem_size *= 1
    elif elem_type == 3:
        mem_size *= 1
    elif elem_type == 4:
        mem_size *= 2
    elif elem_type == 5:
        mem_size *= 2
    elif elem_type == 6:
        mem_size *= 4
    elif elem_type == 7:
        mem_size *= 8
    elif elem_type == 9:
        mem_size *= 1
    elif elem_type == 10:
        mem_size *= 2
    elif elem_type == 11:
        mem_size *= 8
    elif elem_type == 12:
        mem_size *= 4
    elif elem_type == 13:
        mem_size *= 8
    elif elem_type == 14:
        mem_size *= 8
    elif elem_type == 15:
        mem_size *= 16
    else:
        print("Undefined data type")

    return mem_size

def get_bandwidth(graph: xpb2.GraphProto):
    try:
        mem_BW_list = []
        total_mem_BW = 0
        unknown_tensor_list = []
        # traverse all the nodes
        for nodeProto in graph.node:
            # init variables
            read_mem_BW_each_layer = 0
            write_mem_BW_each_layer = 0
            total_each_layer = 0
            # traverse all input tensor
            for input_name in nodeProto.input:
                # get the TensorProto/ValueInfoProto by searching its name
                proto, type_Num = get_valueproto_or_tensorproto_by_name(
                    input_name, graph)
                # parse the ValueInfoProto/TensorProto
                if proto:
                    if type_Num == 3:
                        dtype = getattr(proto, 'data_type', False)
                        # get the shape of the tensor
                        shape = getattr(proto, 'dims', [])
                    elif type_Num == 1 or type_Num == 2:
                        name, dtype, shape_str = _parse_element(proto)
                        shape_str = shape_str.strip('[]')
                        shape_str = shape_str.split(',')
                        shape = []
                        for dim in shape_str:
                            shape.append(int(dim))
                    else:
                        print(
                            '[ERROR MASSAGE] [get_info/mem_BW_without_buf] The Tensor: ',
                            input_name, ' is from a wrong list !')
                else:
                    print(
                        '[ERROR MASSAGE] [get_info/mem_BW_without_buf] The Tensor: ',
                        input_name, ' is no found !')
                    unknown_tensor_list.append(
                        (nodeProto.name, input_name, nodeProto.op_type))
                # calculate the tensor size in btye
                
                read_mem_BW_each_layer += cal_tensor_mem_size(dtype, shape)

            # traverse all output tensor
            for output_name in nodeProto.output:
                # get the TensorProto/ValueInfoProto by searching its name
                proto, type_Num = get_valueproto_or_tensorproto_by_name(
                    output_name, graph)
                # parse the ValueInfoProto
                if proto:
                    if type_Num == 2 or type_Num == 4:
                        # name, dtype, shape = utils._parse_ValueInfoProto(proto)
                        name, dtype, shape_str = _parse_element(proto)
                        shape_str = shape_str.strip('[]')
                        shape_str = shape_str.split(',')
                        shape = []
                        for dim in shape_str:
                            shape.append(int(dim))
                    else:
                        print(
                            '[ERROR MASSAGE] [get_info/mem_BW_without_buf] The Tensor: ',
                            output_name, ' is from a wrong list !')
                else:
                    print(
                        '[ERROR MASSAGE] [get_info/mem_BW_without_buf] The Tensor: ',
                        input_name, ' is no found !')
                    unknown_tensor_list.append(
                        (nodeProto.name, output_name, nodeProto.op_type))
                # calculate the tensor size in btye
                write_mem_BW_each_layer += cal_tensor_mem_size(dtype, shape)

            # cal total bw
            total_each_layer = read_mem_BW_each_layer + write_mem_BW_each_layer

            # store into tuple
            temp_tuple = (nodeProto.name, read_mem_BW_each_layer,
                        write_mem_BW_each_layer, total_each_layer)
            #append it
            mem_BW_list.append(temp_tuple)
            # accmulate the value
            total_mem_BW += total_each_layer

        # display the mem_bw of eahc layer
        columns = ['layer', 'read_bw', 'write_bw', 'total_bw']
        # resort the list
        mem_BW_list = sorted(mem_BW_list,
                             key=lambda Layer: Layer[1],
                             reverse=True)
        print(tabulate(mem_BW_list, headers=columns))
        print(
            '====================================================================================\n'
        )
        # display it
        print(
            "The memory bandwidth for processor to execute a whole model without on-chip-buffer is: \n",
            total_mem_BW, '(bytes)\n',
            float(total_mem_BW) / float(1000000), '(MB)\n')
        # display the unknown tensor
        columns = ['op_name', 'unfound_tensor', 'op_type']
        print(tabulate(unknown_tensor_list, headers=columns))
        print(
            '====================================================================================\n'
        )
    except Exception as e:
        print("[ERROR MASSAGE] Unable to display: " + str(e))
        return False

    return True

#從這裡開始
print("start")
get_bandwidth(inferred_model.graph)

start
layer                            read_bw    write_bw    total_bw
-----------------------------  ---------  ----------  ----------
/classifier/classifier.1/Gemm  151048192       16384   151064576
/classifier/classifier.4/Gemm   67141632       16384    67158016
/classifier/classifier.6/Gemm   16404384        4000    16408384
/features/features.8/Conv        3799552      173056     3972608
/features/features.6/Conv        2785536      259584     3045120
/features/features.10/Conv       2533376      173056     2706432
/features/features.3/Conv        1416192      559872     1976064
/features/features.1/Relu         774400      774400     1548800
/features/features.2/MaxPool      774400      186624      961024
/features/features.0/Conv         695296      774400     1469696
/features/features.4/Relu         559872      559872     1119744
/features/features.5/MaxPool      559872      129792      689664
/features/features.7/Relu         259584      259584      519168
/features/features.

True

### activation memory storage requirement

In [18]:
for node in inferred_model.graph.input:
    names = node.name.split('/')
    node_name = node.name
    dim_values = [dim.dim_value for dim in node.type.tensor_type.shape.dim if dim.dim_value is not None]
    # Print the size of each node
    activations = 1
    for dim in dim_values:
        activations *= dim
    print(f"'{names[-1]}': {dim_values}, activations = {activations}")
    
for node in inferred_model.graph.value_info:
    names = node.name.split('/')
    node_name = node.name
    dim_values = [dim.dim_value for dim in node.type.tensor_type.shape.dim if dim.dim_value is not None]
    # Print the size of each node
    activations = 1
    for dim in dim_values:
        activations *= dim
    print(f"'{names[-1]}': {dim_values}, activations = {activations}")

for node in inferred_model.graph.output:
    names = node.name.split('/')
    node_name = node.name
    dim_values = [dim.dim_value for dim in node.type.tensor_type.shape.dim if dim.dim_value is not None]
    # Print the size of each node
    activations = 1
    for dim in dim_values:
        activations *= dim
    print(f"'{names[-1]}': {dim_values}, activations = {activations}")

'actual_input_1': [1, 3, 224, 224], activations = 150528
'Conv_output_0': [1, 64, 55, 55], activations = 193600
'Relu_output_0': [1, 64, 55, 55], activations = 193600
'MaxPool_output_0': [1, 64, 27, 27], activations = 46656
'Conv_output_0': [1, 192, 27, 27], activations = 139968
'Relu_output_0': [1, 192, 27, 27], activations = 139968
'MaxPool_output_0': [1, 192, 13, 13], activations = 32448
'Conv_output_0': [1, 384, 13, 13], activations = 64896
'Relu_output_0': [1, 384, 13, 13], activations = 64896
'Conv_output_0': [1, 256, 13, 13], activations = 43264
'Relu_output_0': [1, 256, 13, 13], activations = 43264
'Conv_output_0': [1, 256, 13, 13], activations = 43264
'Relu_output_0': [1, 256, 13, 13], activations = 43264
'MaxPool_output_0': [1, 256, 6, 6], activations = 9216
'AveragePool_output_0': [1, 256, 6, 6], activations = 9216
'Flatten_output_0': [1, 9216], activations = 9216
'Gemm_output_0': [1, 4096], activations = 4096
'Relu_output_0': [1, 4096], activations = 4096
'Gemm_output_0': [